In [1]:
from selenium.webdriver import Firefox
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support import expected_conditions as expected
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
from IPython.display import display,Image

import sys


In [74]:
if __name__ == "__main__":
    options = Options()
    options.add_argument('-headless')
    #options.add_argument('cookie:CONSUMERCHOICE_SESSION=t; CONSUMERCHOICE=us/en_us; nike_locale=us/en_us; NIKE_COMMERCE_COUNTRY=US; NIKE_COMMERCE_LANG_LOCALE=en_US;')
    driver = Firefox(executable_path='geckodriver', firefox_options=options)
    wait = WebDriverWait(driver, timeout=60)
    driver.get("https://www.nike.com/language_tunnel");
    #wait.until(expected.presence_of_element_located((By.CSS_SELECTOR, 'span.lang-tunnel__region-img-wrapper')))
    #driver.find_element_by_css_selector('button[data-region="n-america"]').click()
    wait.until(expected.presence_of_element_located((By.CSS_SELECTOR, 'a.lang-tunnel__country-link.js-countryLink')))
    wait.until(expected.presence_of_element_located((By.CSS_SELECTOR, 'button[data-region="n-america"]'))).click()
    #driver.find_element_by_css_selector('button[data-region="n-america"]').click()
    wait.until(expected.visibility_of_element_located((By.CSS_SELECTOR, 'a[data-country="US"]'))).click()
    wait.until(expected.visibility_of_element_located((By.CSS_SELECTOR, 'a.gnav-bar--section-tab')))
    #driver.find_element_by_css_selector('div[data-region="n-america"]').click()
    #driver.get('https://www.nike.com/t/air-vapormax-flyknit-womens-running-shoe-LAPV8l/849557-605')
    #wait.until(expected.visibility_of_element_located((By.NAME, 'q'))).send_keys('headless firefox' + Keys.ENTER)
    #wait.until()
    #wait.until(expected.presence_of_element_located((By.CSS_SELECTOR, 'h1.productTitle')))
    print driver.save_screenshot("1.png")
    #print(driver.page_source)
    driver.quit()

True



### Keywords
**load:** [url]`加载链接`
 
**wait:** [map] `等待`
 * type:
  * presence `存在元素`
  * visibility `可视元素`
  * time `时间`
 * ele:
  * [css selector](https://www.w3schools.com/cssref/css_selectors.asp) `选取单个元素`
 * eles:
  * [css selector](https://www.w3schools.com/cssref/css_selectors.asp) `选取多个元素`
 * millis:
  * 1000 `毫秒数`
  
**loop:** `循环`
 * 内置元素 \$e`迭代元素` \$i`迭代下标`
 * **in:** [map] `元素集合`
  * eles
 * **each:** `循环操作`
 
**click:** [map] [\$e] `点击元素`
 * ele
  
**select** [map] `选取元素文本写入path`
 * ele
 * path:
  * /product/sku[]/price
 * attr:
     * `属性值`
 * regex:
     * `正则表达式`

**if:** [expression]
 * **then:**
 * **elif:** [expression]
  * **then:**
 * **else:**
 
**put:** `设值`
 * **path** 
 * **value**
 
**append** `追加值`
 * **path** 
 * **value**
 
**back:** `回退`
 
**fill:** `填充`
 * **ele**
 * **value**


In [96]:
import yaml
_Config = """
- load: http://www.perfumesclub.cn/1001910.html
- wait: {type: presence,ele: h1.fn.title-name}  
- select: {ele: h1.fn.title-name, path: /product/title}     
- loop:
    in: {eles: "ul.property-list.clearfix > li"}
    each:
        - click: $e
"""

# - select: {ele: a.product-brand, path: /product/brand, regex: '/(.*)\\.html'}
# - select: {ele: 'div.tab-content > p:nth-child(1)', path: /product/desc}
# - select: {ele: 'div.tab-content > p:nth-child(2)', append: /product/desc}
# - loop: 
#     in: {eles: "div[data-attr='color'] > a"} 
#     each:
#     - click: $e
#     - wait: {type: time,millis: 500}
#     - select: {ele: div.price > span.sales, path: /tmp/price} 
#     - select: {ele: button.add-to-cart}
#     - if: $1=="SOLD OUT"
#     - then:
#         - set: {path: /tmp/store, value: 0}
#     - else:
#         - set: {path: /tmp/store, value: 100}
#     - put: {path: "/product/sku[]", value: $(/tmp)}


INPUT = yaml.load(_Config)
print INPUT

[{'load': 'http://www.perfumesclub.cn/1001910.html'}, {'wait': {'type': 'presence', 'ele': 'h1.fn.title-name'}}, {'select': {'path': '/product/title', 'ele': 'h1.fn.title-name'}}, {'loop': {'in': {'eles': 'ul.property-list.clearfix > li'}, 'each': [{'click': '$e'}]}}]


### Expression


In [85]:
from ctypes import *
libast = cdll.LoadLibrary("./libcondast.so")

class Exp:
    LIB_AST = None
    FUNC_P = None
    
    @staticmethod
    def install(lib):
        Exp.LIB_AST = lib
        Exp.FUNC_P = lib.calc_exp
        Exp.FUNC_P.restype = c_double
    
    @staticmethod
    def calc(exp):
        r = Exp.FUNC_P(exp)
        if str(r).split(".")[1] == "0":
            r = int(r)
        return r

Exp.install(libast)
print Exp.calc("1+2+3")


6


### Path


In [55]:
import re
import types
class PathQuery:
    def __init__(self):
        self.type = None
        self.key = None
        self.next = None
        self.it_exp = None
        self.end = False
        self.list_all = False
        
    def desc(self):
        t = self
        s = ""
        while t != None:
            s+= "type="+t.type+"&key="+t.key+" "
            t = t.next
        return s
        
class Path:
    ROOT={}
    _PATH_VAR_MAP = {"/":ROOT}
    INNER_VAR_PREFIX = "__"
    @staticmethod
    def set_value(path,value):
        Path.set_value_by_query(Path.new_query(path),value)
        
    @staticmethod
    def get_value(path):
        return Path.get_value_by_query(Path.new_query(path))
    
    @staticmethod
    def set_value_by_query(query,value):
        cur_var = Path._PATH_VAR_MAP
        q = query
        while q != None:
            if q.end:
                if q.type == "list":
                    if q.it_exp == "":
                        if q.list_all:
                            if type(value) is not types.ListType:
                                raise ValueError("The type of value should be `List`.")
                            cur_var[q.key] = value
                        else:
                            cur_var[q.key].append(value)
                    else:
                        cur_var[q.key][Exp.calc(q.it_exp)] = value
                elif q.type == "map":
                    if type(value) is not types.DictType:
                        raise ValueError("The type of value should be `Dict`.")
                    cur_var[q.key] = value
                else:
                    if type(value) is types.ListType or type(value) is types.DictType:
                        raise ValueError("Can not set the value of type `Dict` or `List` to node.")
                    cur_var[q.key] = value
            else:
                if q.type == "list":
                    if q.key not in cur_var:
                        cur_var[q.key] = []
                    if q.it_exp == "":
                        cur_var = cur_var[q.key][-1]
                    else:
                        cur_var = cur_var[q.key][Exp.calc(q.it_exp)]
                else:
                    if q.key not in cur_var:
                        cur_var[q.key] = {}
                    cur_var = cur_var[q.key]
            q = q.next
    
    @staticmethod
    def get_value_by_query(query):
        cur_var = Path._PATH_VAR_MAP
        q = query
        while q != None:
            if q.end:
                if q.it_exp != "" and q.it_exp != None:
                    return cur_var[q.key][Exp.calc(q.it_exp)]
                else:
                    return cur_var[q.key]
            else:
                if q.type == "list":
                    if q.it_exp == "":
                        cur_var = cur_var[q.key][-1]
                    else:
                        cur_var = cur_var[q.key][Exp.calc(q.it_exp)]
                else:
                    cur_var = cur_var[q.key]
            q = q.next
        
    @staticmethod
    def new_query(path):
        l = 1
        p_len = len(path)
        if p_len < l:
            return None
        
        p = re.compile(r'.*\[(.*)\]')
        pq = PathQuery()
        builder = ""
        for c in path:
            builder += c
            if c == '/':
                if builder == "/":
                    pq.type = "root"
                    pq.key = "//"
                else:
                    m = p.match(builder)
                    if m == None:
                        pq.type = "map"
                        pq.key = builder
                    else:
                        pq.type = "list"
                        pq.it_exp = m.group(1)
                        if pq.it_exp == "":
                            pq.key = builder.replace('[]','')
                        else:
                            pq.key = builder.replace('['+m.group(1)+']','')
                pq.key = pq.key[0:len(pq.key)-1] #remove the last '/'
                pq.next = Path.new_query(path[l:p_len])
                if pq.next == None:
                    if pq.type == "list":
                        pq.list_all = True
                    pq.end = True
                return pq
            elif l == len(path):
                m = p.match(builder)
                if m == None:
                    pq.type = "node"
                    pq.key = builder
                else:
                    pq.type = "list"
                    pq.it_exp = m.group(1)
                    if pq.it_exp == "":
                        pq.key = builder.replace('[]','')
                    else:
                        pq.key = builder.replace('['+m.group(1)+']','')
                pq.end = True
                pq.next = Path.new_query("")
                return pq
            l+=1

### Usage Of Path
#### 设置读取属性值

In [3]:
Path.set_value("/node",100)
print Path.get_value("/node")

100


#### 设置读取字典

In [88]:
Path.set_value("/map/",{"p1":1,"p2":2})
#如果设置map字典值已经设置，"/map/" 或者 “/map” 都将读取该字典的值
print Path.get_value("/map/")  
#字典追加
Path.set_value("/map/p3",3)
print Path.get_value("/map/")
print Path.get_value("/map/p3")

{'p2': 2, 'p1': 1}
{'p2': 2, 'p3': 3, 'p1': 1}
3


#### 设置读取数组

In [5]:
Path.set_value("/list[]/",[1,2])
print Path.get_value("/list[]/")
#数组追加
Path.set_value("/list[]",3)
print Path.get_value("/list[]/")
#数组访问
print Path.get_value("/list[-1]") #末尾元素
print Path.get_value("/list[0]") 
print Path.get_value("/list[0+1]") #表达式

[1, 2]
[1, 2, 3]
3
1
2


### Var

In [74]:
class Var:
    def __init__(self,s):
        if s[0] =="L":
            self.func = "L"
            self.value = s[1:]
        else:
            self.func = ""
            self.value = s
    
    def eval(self):
        if self.value == "e" or self.value == "i":
            return Path.get_value("/"+Path.INNER_VAR_PREFIX+self.value)
        else:
            v = Path.get_value(self.value)
            if self.func == "L":
                return str(len(v))
            return str(v)

### Variable Picker

In [90]:
class VarPicker:
    STAT_SCAN = 0
    STAT_DOLLAR = 1
    @staticmethod
    def scan(s):
        stat = VarPicker.STAT_SCAN
        cap = ""
        r = str(s)
        for i in range(0,len(s)):
            c = s[i]
            if stat == VarPicker.STAT_SCAN:
                if c == "$":
                    stat = VarPicker.STAT_DOLLAR
            elif stat == VarPicker.STAT_DOLLAR:
                if c >= "0" and c <="9" or c>="a" and c <"z" or c>="A" and c<="Z" or c =="/" or c=="[" or c=="]":
                    cap += c
                else:
                    r = r.replace("$"+cap,Var(cap).eval(),1)
                    cap = ""
                    stat = VarPicker.STAT_SCAN

        if stat == VarPicker.STAT_DOLLAR: #/test
            r = r.replace("$" + cap, Var(cap).eval(),1)
        return r

# Path.set_value("/__e",123)
# print VarPicker.scan("$e")
Path.set_value("/test","10")
#put: {path:"/a",value:"$/b"}
print VarPicker.scan("$L/test")

2


### GAtom Super Class

In [6]:
class GAtom:
    snapshot_icr = 0
    def __init__(self,token,wk=None):
        self.token = token
        self.wk = wk
        self.MAP = {}
        
    def snapshot(self):
        file_name = "snapshot"+str(GAtom.snapshot_icr)+".png"
        GAtom.snapshot_icr+=1
        self.wk.driver.save_screenshot(file_name)
        display(Image(file_name))
        
    def path_setter(self,path,value):
        Path.set_value(path,value)
    
    def path_getter(self,path):
        return Path.get_value(path)

### Load

In [94]:
class Load(GAtom,object):
    def __init__(self,url,wk=None):
        super(Load,self).__init__("load",wk)
        self.url = url
        
    def action(self):
        print "load..."
        self.wk.driver.get(self.url)
        #super(Load,self).snapshot()
        

l = Load("http://www.coach.com/coach-swagger-27/12117.html?dwvar_color=DKLIQ")
print l.token+":"+l.url

load:http://www.coach.com/coach-swagger-27/12117.html?dwvar_color=DKLIQ


### Wait

In [8]:
class Wait(GAtom,object):
    def __init__(self,dict,wk=None):
        super(Wait,self).__init__("wait",wk)
        if "ele" not in dict:
            raise AttributeError("token["+self.token+"] must have `ele` attr.")
        self.ele = dict["ele"]
        
        if "type" not in dict:
            raise AttributeError("token["+self.token+"] must have `type` attr.")
        self.type = dict["type"]
        
        self.millis = "none"
        if "millis" in dict:
            self.millis = dict["millis"]
        
            
    def action(self):
        print "wait..."
        if type == "presence":
            self.wk.wait.until(expected.presence_of_element_located((By.CSS_SELECTOR, self.ele)))

w = Wait({'type': 'presence', 'ele': 'div.price'})
print w.token+":"+"ele->"+w.ele+",type->"+w.type+",millis->"+w.millis

wait:ele->div.price,type->presence,millis->none


### Select

In [9]:
class Select(GAtom,object):
    def __init__(self,dict,wk=None):
        super(Select,self).__init__("select",wk)
        if "ele" not in dict:
            raise AttributeError("token["+self.token+"] must have `ele` attr.")
        self.ele = dict["ele"]
        
        self.path = "none"
        if "path" in dict:
            self.path = dict["path"]
        
        self.append = "none"
        if "append" in dict:
            self.append = dict["append"]
        
        self.attr = "none"
        if "attr" in dict:
            self.attr = dict["attr"]
            
    def action(self):
        print "select..."
        Path.set_value(self.path,self.wk.driver.find_element_by_css_selector(self.ele).text)
        
        
s = Select({'path': '/product/title', 'ele': 'h.product-name-desc'})
print s.token+":"+"path->"+s.path+",ele->"+s.ele+",attr->"+s.attr

select:path->/product/title,ele->h.product-name-desc,attr->none


### Click

In [80]:
class Click(GAtom,object):
    def __init__(self,dict,wk=None):
        super(Click,self).__init__("click",wk)
        
    def action(self):
        Path.get_value("/"+Path.INNER_VAR_PREFIX+"e").click()
        super(Click,self).snapshot()
        print self.wk.driver.current_url

### Loop

In [63]:
class Loop(GAtom,object):
    def __init__(self,dict,wk=None):
        super(Loop,self).__init__("loop",wk)
        
        if "in" not in dict:
            raise AttributeError("token["+self.token+"] must have `in` attr.")
        self.eles = dict["in"]["eles"]
        
        if "each" not in dict:
            raise AttributeError("token["+self.token+"] must have `each` attr.")
        self.each = dict["each"]
        
        self.cmds = []
        for atom in self.each:
            if "wait" in atom:
                self.cmds.append(Wait(atom["wait"],wk))
            if "select" in atom:
                self.cmds.append(Select(atom["select"],wk))
            if "click" in atom:
                self.cmds.append(Click(atom["click"],wk))
        
    def action(self):
        print "loop..."
        s_eles = self.wk.driver.find_elements_by_css_selector(self.eles)
        print self.eles
        print s_eles
        for i in range(0,len(s_eles)):
            s_ele = s_eles[i]
            Path.set_value("/"+Path.INNER_VAR_PREFIX+"e",s_ele)
            Path.set_value("/"+Path.INNER_VAR_PREFIX+"i",i)
            for cmd in self.cmds:
                print cmd
                cmd.action()
             
            

### VM

In [95]:
# coding: utf-8  
from selenium.webdriver import Firefox
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support import expected_conditions as expected
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
from IPython.display import display,Image

options = Options()
options.add_argument('-headless')
profile = FirefoxProfile()
#profile.set_preference('permissions.default.stylesheet', 2)
profile.set_preference('permissions.default.image', 2)
profile.set_preference('dom.ipc.plugins.enabled.libflashplayer.so', 'false')
driver = Firefox(executable_path='geckodriver', firefox_options=options,firefox_profile=profile)
#driver = Chrome(executable_path='chromedriver', )

class WebKernel:
    def __init__(self,driver):
        self.driver = driver
        self.wait = WebDriverWait(driver, timeout=60)
        
wk = WebKernel(driver)
        

def vm(arr):
    cmds=[]
    for atom in arr:
        if "load" in atom:
            cmds.append(Load(atom["load"],wk))
        if "wait" in atom:
            cmds.append(Wait(atom["wait"],wk))
        if "select" in atom:
            cmds.append(Select(atom["select"],wk))
#         if "loop" in atom:
#             cmds.append(Loop(atom["loop"],wk))
    return cmds


cmds = vm(INPUT)

def exe(cmds):
    for cmd in cmds:
        cmd.action()

exe(cmds)
print Path.get_value("/product/title")

load...
wait...
select...
ARTDECO 雅蔻 完美颜色口红 4g #18 innocent kiss
